# WeirdMachine

> Misc | 971 Points | 10 Solves
> 
> Author: zeyu2001
> 
> You're interviewing for a CS major at Hogwarts.
> 
> "Here at Hogwarts, our computers are a bit... different. Can you help us write a few programs on our WeirdMachine™?"
> 
> Architecture:
> 
> The WeirdMachine is a collection of up to 10 smaller computers, each with 10 registers storing values from -1000 to 1000.
> At the beginning of the program, we start off with only 1 computer, and its first two registers R0 and R1 are loaded with the program inputs.
> Instructions:
> 
> SET x y: Set Rx to the value y.  
> ADD x y: Set Rx to the value of Rx + Ry.  
> NEG x: Clone the current computer, and add this clone to the collection of computers within the WeirdMachine. In the cloned computer, Rx is multiplied by -1.  
> JIZ x y: If Rx == 0, jump to instruction y. Instructions start from index 0.  
> JNZ x y: If Rx != 0, jump to instruction y. Instructions start from index 0.  
> HALT x: Stop the program. The output of the program is the value of Rx.  
> All cloned computers as a result of NEG will run in parallel (i.e. run one instruction per tick). The first computer that HALTs will return the answer to the entire program, regardless of whether other computers are still running.
> 
> The challenge: Write a script that performs R0 * R1.
> 
> nc fun.chall.seetf.sg 20001
> 
> Attachment: [misc_weirdmachine.zip](attachments/misc_weirdmachine.zip)  
> MD5: 9765bcff9f79eaab765d08b8ceb0af02

I wrote an assembler for this with named registers, comments and labels. It's overkill.

Let's rename inputs from `R0` to `a` and `R1` to `b`.

Our only way to get negative numbers is the `NEG` operation, and after that the machine is forked into two instances per `NEG` operation. Specifically for the scenario where both `a` and `b` are negative, we need to negate at least one of their numbers, then return the value obtained from repeat addition.

The main observation to make is that we can check if a machine is on the negated or non-negated fork using two extra registers:

```
; input in register r_in
set r_in 0
add r_tmp r_in
neg r_tmp
set r_acc 0
add r_acc r_in
add_r_acc r_tmp
jiz r_acc ?
```

`r_acc` will have 0 if in the negated fork, otherwise it will be double that of `r_in`. We can do this for both `a` and `b`, so that at the end we get four forks which may or may not have extra registers storing `ea = -a` and `eb = -b`.

We can assign each fork to a respective adding loop. An adding loop that decrements `a` until it reaches 0 will only halt if `a` was positive to begin with, and vice versa for `b`. One way to assign it is as below.

The last scenario we need to consider is if both `a` and `b` are negative. We can use `ea` and `eb` in this scenario which already stores `-a` and `-b` respectively. The implementation below decrements `eb`, thus halting if `eb` is positive and therefore if `b` is negative.

| Program        | ea+ eb+ | ea+ eb- | ea- eb+ | ea- eb- |
|----------------|---------|---------|---------|---------|
| Halt condition | b+      | a+      | b+      | b-     |
| Input: a+ b+   | ✔️       | ✔️       | ✔️       |         |
| Input: a+ b-   |         | ✔️       |         | ✔️       |
| Input: a- b+   | ✔️       |         | ✔️       |         |
| Input: a- b-   |         |         |         | ✔️       |

In [9]:
replaces = [
  ("r_a", "0"),
  ("r_b", "1"),
  ("r_one", "2"),
  ("r_neg", "3"),
  ("r_na", "4"),
  ("r_nb", "5"),
  ("r_ea", "6"),
  ("r_eb", "7"),
  ("r_res", "8"),
  ("r_temp", "9")
]

def asm(program):
  program = program.strip().upper()
  for a, b in replaces:
    program = program.replace(a.upper(), b.upper())

  lines = []
  markers = {}
  line_counter = 0
  for line in program.split("\n"):
    line, _, comment = line.strip().partition(";")
    line = line.strip()
    if not line:
      continue
    if line[0] == ":":
      markers[line] = str(line_counter)
    else:
      lines.append(line)
      line_counter += 1

  all_lines = "\n".join(lines)
  for k, v in markers.items():
    all_lines = all_lines.replace(k, v)
  return all_lines

In [10]:
program = f"""
:init
  set r_one 1;
  set r_neg -1;
  set r_na 1;
  set r_nb 1;
  set r_ea 0;
  set r_eb 0;
  add r_ea r_a;
  add r_eb r_b;
  set r_res 0;

:split
  neg r_ea;
  neg r_eb;

  set r_na 0;
  add r_na r_a;
  add r_na r_ea;

  set r_nb 0;
  add r_nb r_b;
  add r_nb r_eb;

  jiz r_na :-a?b
:+a?b
  jiz r_nb :+a-b
:+a+b
  jnz r_one :halt_if_a+;
:+a-b
  jnz r_one :halt_if_b+;
:-a?b
  jiz r_nb :-a-b
:-a+b
  jnz r_one :halt_if_a+;
:-a-b
  jnz r_one :use_neg

:halt_if_a+
  add r_res r_a;
  add r_b r_neg;
  jnz r_b :halt_if_a+;
  halt r_res;

:halt_if_b+
  add r_res r_b;
  add r_a r_neg;
  jnz r_a :halt_if_b+;
  halt r_res;

:use_neg
  add r_res r_ea;
  add r_eb r_neg;
  jnz r_eb :use_neg;
  halt r_res;
"""

assembled = asm(program)
print(assembled)

SET 2 1
SET 3 -1
SET 4 1
SET 5 1
SET 6 0
SET 7 0
ADD 6 0
ADD 7 1
SET 8 0
NEG 6
NEG 7
SET 4 0
ADD 4 0
ADD 4 6
SET 5 0
ADD 5 1
ADD 5 7
JIZ 4 21
JIZ 5 20
JNZ 2 24
JNZ 2 28
JIZ 5 23
JNZ 2 24
JNZ 2 32
ADD 8 0
ADD 1 3
JNZ 1 24
HALT 8
ADD 8 1
ADD 0 3
JNZ 0 28
HALT 8
ADD 8 6
ADD 7 3
JNZ 7 32
HALT 8


In [11]:
from pwn import *

r = remote("fun.chall.seetf.sg", 20001)
r.recvuntil(b"ENTER YOUR SCRIPT")
r.recvline()
r.sendline((assembled + "\n").encode())
res = r.recvall()
print(res.decode())

[x] Opening connection to fun.chall.seetf.sg on port 20001
[x] Opening connection to fun.chall.seetf.sg on port 20001: Trying 34.131.197.225
[+] Opening connection to fun.chall.seetf.sg on port 20001: Done
[x] Receiving all data
[x] Receiving all data: 0B
[x] Receiving all data: 175B
[x] Receiving all data: 203B
[x] Receiving all data: 230B
[x] Receiving all data: 231B
[x] Receiving all data: 259B
[x] Receiving all data: 288B
[x] Receiving all data: 323B
[x] Receiving all data: 443B
[+] Receiving all data: Done (443B)
[*] Closed connection to fun.chall.seetf.sg port 20001
========== RESULT ===========
Result: -25
============ END ============
Sample test case passed. Good job! Beginning to evaluate real test cases.
========== TESTING ==========
Range: (0, 1000) (0, 1000)
Range: (-1000, 0) (0, 1000)
Range: (0, 1000) (-1000, 0)
Range: (-1000, 0) (-1000, 0)
Range: (-1000, 1000) (-1000, 1000)
All test cases passed. Good job!
Here's your flag: SEE{und3r6r4d_4dm15510n5_4r3_cr4zy_7fc37a510e35